# Random Forest Model for classifying Gender by Indonesian names

## Import necessary files needed

In [1]:
import joblib
from random_forest_cart import RandomForestCART
from random_forest_cart import CARTTree
# Load the trained model and preprocessing tools
rf = joblib.load("random_forest_cart.pkl")
vectorizer = joblib.load("vectorizer.pkl")
selector = joblib.load("selector.pkl")
scaler = joblib.load("scaler.pkl")

## Two option for model that needed to be run first:
1. Retrieve and Inserting Data from SQL server
2. Retrieve the data from an CSV and save it into a CSV

### OPTION 1: Retrieve and Inserting data from SQL server

In [ ]:
import pyodbc
import pandas as pd

# Define the function to fetch and preprocess data
def preprocessing_dataset():
    print("Connecting to SQL Server...")
    try:
        # Define connection string (replace with your actual SQL Server details)
        conn = pyodbc.connect(
            "DRIVER={SQL Server};"
            "SERVER=10.12.30.240;"
            "DATABASE=GORPDWHBI;"  # Replace with your database name
            "UID=viewer;"
            "PWD=viewer1;"
        )
        print("Successfully connected to SQL Server!")

        # Query to fetch the required columns
        query = """
        WITH RankedCustomers AS (
            SELECT 
                MyValueId, 
                CustomerName, 
                Gender,
                ROW_NUMBER() OVER (PARTITION BY CustomerName ORDER BY MyValueId ASC) AS row_num
            FROM dbo.DimCustomer
            WHERE Gender NOT IN ('F', 'M')
              AND MyValueId IS NOT NULL 
              AND CustomerName IS NOT NULL
              AND LTRIM(RTRIM(MyValueId)) <> ''  
              AND LTRIM(RTRIM(CustomerName)) <> ''  
        )
        SELECT MyValueId, CustomerName, Gender 
        FROM RankedCustomers
        WHERE row_num = 1;
         """
        print("Processing query...")

        # Read the data from SQL Server
        GenderData = pd.read_sql(query, conn)
        print("Successfully fetched data from SQL Server!")

    except Exception as e:
        print(f"Failed to connect or fetch data: {e}")
        return None
    finally:
        # Close connection
        conn.close()
        print("Connection closed.")

    # Drop null values in 'CustomerName'
    print("Cleaning data...")
    GenderData = GenderData.dropna(subset=['CustomerName'])
    print("Data successfully cleaned!")

    # Function to split names
    def split_name(name):
        name = name.lower()
        parts = name.split()
        
        first_name = second_name = third_name = fourth_name = last_name = ' '
        
        if len(parts) >= 1:
            first_name = parts[0]
        if len(parts) >= 2:
            second_name = parts[1]
        if len(parts) >= 3:
            third_name = parts[2]
        if len(parts) >= 4:
            fourth_name = parts[3]
        if len(parts) >= 5:
            last_name = ' '.join(parts[4:])
        
        return pd.Series([first_name, second_name, third_name, fourth_name, last_name])
    
    # Apply function to DataFrame
    print("Splitting customer names into individual parts...")
    GenderData[['FirstName', 'SecondName', 'ThirdName', 'FourthName', 'LastName']] = GenderData['CustomerName'].apply(split_name)
    print("Name splitting completed!")

    # Drop original 'CustomerName' and 'Gender' columns
    columns_to_drop = ['CustomerName', 'Gender']
    GenderData = GenderData.drop(columns=columns_to_drop, errors='ignore')
    
    print("Preprocessing completed successfully!")

    return GenderData

In [ ]:
import time
import pandas as pd
import numpy as np
from datetime import datetime

def run_prediction():
    print("Starting the prediction process...")

    # Log execution time
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"Script executed at: {timestamp}")

    try:
        # Fetch and preprocess data
        print("Fetching and preprocessing dataset from SQL Server...")
        GenderData = preprocessing_dataset()
        if GenderData is None or GenderData.empty:
            print("No data retrieved. Exiting process.")
            return
        print("Data fetched and preprocessed successfully!")

        def get_bigrams(name):
            return ' '.join(name[i:i+2] for i in range(len(name)-1)) if isinstance(name, str) else ""


        # Join all name columns into a single column
        X_new = GenderData[['FirstName', 'SecondName', 'ThirdName', 'FourthName', 'LastName']].astype(str)
        GenderData["CustomerName"] = X_new.apply(lambda row: ' '.join(row).strip(), axis=1)

        # Extract bigrams
        print("Extracting bigrams...")
        X_new = X_new.map(get_bigrams)
        X_new["FullName"] = X_new.apply(lambda row: ' '.join(row), axis=1)
        print("Bigrams extracted!")

        # Apply HashingVectorizer
        print("Applying feature transformation...")
        X_new_hashed = vectorizer.transform(X_new["FullName"])  
        X_new_df = pd.DataFrame(X_new_hashed.toarray())
        print("Feature transformation completed!")

        # Remove low-variance features
        print("Removing low-variance features...")
        X_new_df = pd.DataFrame(selector.transform(X_new_df))
        print("Low-variance features removed!")

        # Scale features
        print("Scaling features...")
        X_new_scaled = scaler.transform(X_new_df)
        print("Feature scaling completed!")

        # Predict using the trained model
        print("Model is predicting genders...")
        y_pred_new = rf.predict(X_new_scaled)
        print("Prediction completed!")

        # Map predictions back to 'M' and 'F'
        print("Mapping predictions to labels...")
        GenderData["Predicted_Gender"] = np.where(y_pred_new == 0, 'M', 'F')
        GenderData = GenderData[["CustomerName", "Predicted_Gender"]]
        print("Mapping completed!")

        # Show preview of mapped data
        print("Preview of predicted data:")
        print(GenderData.head())  # Show first 5 rows
        conn = pyodbc.connect(
            "DRIVER={SQL Server};"
            "SERVER=10.12.30.240;"
            "DATABASE=GORPDWHBI;"
            "UID=viewer;"
            "PWD=viewer1;"
        )
        
        cursor = conn.cursor()
        
        # Ensure no NaN values
        GenderData = GenderData.dropna(subset=["CustomerName", "Predicted_Gender"])
        
        # Loop through DataFrame rows and insert into SQL Server
        print("Inserting predicted data into SQL Server...")
        for index, row in GenderData.iterrows():
            cursor.execute("""
                MERGE INTO dbo.MemberGenderPrediction AS target
                USING (SELECT ? AS CustomerName, ? AS GenderPrediction) AS source
                ON target.CustomerName = source.CustomerName
                WHEN NOT MATCHED THEN
                    INSERT (CustomerName, GenderPrediction) VALUES (source.CustomerName, source.GenderPrediction);
            """, (row["CustomerName"], row["Predicted_Gender"]))

        conn.commit()
        cursor.close()
        conn.close()
        print("Data inserted successfully!")
        
    except Exception as e:
        print(f"An error occurred: {e}")

    print("Prediction process completed!")


### OPTION 2: Retrieve the data from an CSV and save it into a CSV

In [2]:
import pyodbc
import pandas as pd

# Define the function to fetch and preprocess data
def preprocessing_dataset():
    print("Taking data from dataset...")
    try:
        GenderData = pd.read_csv("DatasetGenderNeededClarify.csv")
    except Exception as e:
        print(f"Failed to fetch data: {e}")
        return None
    finally:
        print("Data get!")

    # Drop null values in 'CustomerName'
    print("Cleaning data...")
    GenderData = GenderData.dropna(subset=['CustomerName'])
    print("Data successfully cleaned!")

    # Function to split names
    def split_name(name):
        name = name.lower()
        parts = name.split()
        
        first_name = second_name = third_name = fourth_name = last_name = ' '
        
        if len(parts) >= 1:
            first_name = parts[0]
        if len(parts) >= 2:
            second_name = parts[1]
        if len(parts) >= 3:
            third_name = parts[2]
        if len(parts) >= 4:
            fourth_name = parts[3]
        if len(parts) >= 5:
            last_name = ' '.join(parts[4:])
        
        return pd.Series([first_name, second_name, third_name, fourth_name, last_name])
    
    # Apply function to DataFrame
    print("Splitting customer names into individual parts...")
    GenderData[['FirstName', 'SecondName', 'ThirdName', 'FourthName', 'LastName']] = GenderData['CustomerName'].apply(split_name)
    print("Name splitting completed!")

    # Drop original 'CustomerName' and 'Gender' columns
    columns_to_drop = ['CustomerName', 'Gender']
    GenderData = GenderData.drop(columns=columns_to_drop, errors='ignore')
    
    print("Preprocessing completed successfully!")

    return GenderData

In [3]:
import time
import pandas as pd
import numpy as np
from datetime import datetime

def run_prediction():
    print("Starting the prediction process...")

    # Log execution time
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"Script executed at: {timestamp}")

    try:
        # Fetch and preprocess data
        print("Fetching and preprocessing dataset from csv...")
        GenderData = preprocessing_dataset()
        if GenderData is None or GenderData.empty:
            print("No data retrieved. Exiting process.")
            return
        print("Data fetched and preprocessed successfully!")

        def get_bigrams(name):
            return ' '.join(name[i:i+2] for i in range(len(name)-1)) if isinstance(name, str) else ""

        # Join all name columns into a single column
        X_new = GenderData[['FirstName', 'SecondName', 'ThirdName', 'FourthName', 'LastName']].astype(str)
        GenderData["CustomerName"] = X_new.apply(lambda row: ' '.join(row).strip(), axis=1)

        # Extract bigrams
        print("Extracting bigrams...")
        X_new = X_new.map(get_bigrams)
        X_new["FullName"] = X_new.apply(lambda row: ' '.join(row), axis=1)
        print("Bigrams extracted!")

        # Apply HashingVectorizer
        print("Applying feature transformation...")
        X_new_hashed = vectorizer.transform(X_new["FullName"])  
        X_new_df = pd.DataFrame(X_new_hashed.toarray())
        print("Feature transformation completed!")

        # Remove low-variance features
        print("Removing low-variance features...")
        X_new_df = pd.DataFrame(selector.transform(X_new_df))
        print("Low-variance features removed!")

        # Scale features
        print("Scaling features...")
        X_new_scaled = scaler.transform(X_new_df)
        print("Feature scaling completed!")

        # Predict using the trained model
        print("Model is predicting genders...")
        y_pred_new = rf.predict(X_new_scaled)
        print("Prediction completed!")

        # Map predictions back to 'M' and 'F'
        print("Mapping predictions to labels...")
        GenderData["Predicted_Gender"] = np.where(y_pred_new == 0, 'M', 'F')
        GenderData = GenderData[["MyValueId", "CustomerName", "Predicted_Gender"]]
        print("Mapping completed!")

        # Show preview of mapped data
        print("Preview of predicted data:")
        print(GenderData.head(10))  # Show first 10 rows
        
        # Save output
        output_filename = f"GenderData_Predicted_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        GenderData.head(100).to_csv(output_filename, index=False)
        print(f"Predicted data (first 100 rows) saved successfully as {output_filename}!")

    except Exception as e:
        print(f"An error occurred: {e}")

    print("Prediction process completed!")


## Below code is too start the program!

In [4]:
import time

while True:
    start_prediction_time = time.time()
    run_prediction()
    end_prediction_time = time.time()
    print(f"Testing Time: {end_prediction_time - start_prediction_time:.4f} seconds")
    
    # Sleep for 15 hour (900 seconds)
    
    print("Sleeping for 15 minutes...")
    time.sleep(900)

Starting the prediction process...
Script executed at: 2025-02-10 11:56:32
Fetching and preprocessing dataset from csv...
Taking data from dataset...
Data get!
Cleaning data...
Data successfully cleaned!
Splitting customer names into individual parts...
Name splitting completed!
Preprocessing completed successfully!
Data fetched and preprocessed successfully!
Extracting bigrams...
Bigrams extracted!
Applying feature transformation...
Feature transformation completed!
Removing low-variance features...
Low-variance features removed!
Scaling features...
Feature scaling completed!
Model is predicting genders...
Prediction completed!
Mapping predictions to labels...
Mapping completed!
Preview of predicted data:
  MyValueId                CustomerName Predicted_Gender
0    VJDLSC               aprilia silmi                F
1    VT4RFF          sasta citra fauzia                F
2    VHVSRM                 𝒅𝒆𝒅𝒆 sopiah                F
3    VEUOAU               ade mai putra                M

KeyboardInterrupt: 